# Project 1 

Importing allowed libraries...

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## I. Load the Data

In [34]:
# Different paths to access data file

Louise_Path = '/Users/louiseplacidet/Desktop/Machine Learning/Project 1/Git_ML_P1/Data/'

In [29]:
def load_data():
    """load data."""
    data = np.genfromtxt(Louise_Path+"train.csv", delimiter=",",dtype='str' skip_header=1, usecols = [])
    ind = data[:,0].astype(np.float)
    x = data[:,2:].astype(np.float)
    y = data[:,1]
    return x, y, ind

In [30]:
x, y, ind = load_data()

In [31]:
x

array([[ 138.47 ,   51.655,   97.827, ...,    1.24 ,   -2.475,  113.497],
       [ 160.937,   68.768,  103.235, ..., -999.   , -999.   ,   46.226],
       [-999.   ,  162.172,  125.953, ..., -999.   , -999.   ,   44.251],
       ...,
       [ 105.457,   60.526,   75.839, ..., -999.   , -999.   ,   41.992],
       [  94.951,   19.362,   68.812, ..., -999.   , -999.   ,    0.   ],
       [-999.   ,   72.756,   70.831, ..., -999.   , -999.   ,    0.   ]])

In [32]:
y

array([nan, nan, nan, ..., nan, nan, nan])

In [33]:
ind

array([100000., 100001., 100002., ..., 349997., 349998., 349999.])

## Build Model

In [ ]:
num_samples = len(y)
tx = np.c_[np.ones(num_samples), x]

In [ ]:
tx

# II. Implementation Functions

In [ ]:
def compute_loss_mse(y, tx, w):
    """Calculate the loss using mse."""
    e = y - tx.dot(w)
    return 1/2 * np.mean(e**2)

### Least Squares GD

In [ ]:
def compute_gradient(y, tx, w):
    """Compute a gradient."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    
    return gradient, e

In [ ]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
        """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):

        gradient, e = compute_gradient(y,tx,w)
        loss = compute_loss(y, tx, w)
        
        w = w - gamma * gradient
        # store w and loss
        ws.append(w)
        losses.append(loss)

    return w, loss

### Least Squares SGD

In [ ]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    e = y - tx.dot(w)
    gradient = - (1/len(e)) * tx.T.dot(e)
    return gradient, e

In [ ]:
def batch_iter(y, tx, batch_size, num_batches=1, shuffle=True):
    """
    Generate a minibatch iterator for a dataset.
    Takes as input two iterables (here the output desired values 'y' and the input data 'tx')
    Outputs an iterator which gives mini-batches of `batch_size` matching elements from `y` and `tx`.
    Data can be randomly shuffled to avoid ordering in the original data messing with the randomness of the minibatches.
    Example of use :
    for minibatch_y, minibatch_tx in batch_iter(y, tx, 32):
        <DO-SOMETHING>
    """
    data_size = len(y)

    if shuffle:
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_y = y[shuffle_indices]
        shuffled_tx = tx[shuffle_indices]
    else:
        shuffled_y = y
        shuffled_tx = tx
    for batch_num in range(num_batches):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size)
        if start_index != end_index:
            yield shuffled_y[start_index:end_index], shuffled_tx[start_index:end_index]


In [ ]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    batch_size = 5 ##TBD
    w = initial_w
    
    for n_iter in range(max_iters):
        for y_batch, tx_batch in batch_iter(y,tx,batch_size = batch_size, num_batches = 1):
            gradient , e = compute_stoch_gradient(y_batch,tx_batch, w)
            w = w - gamma*gradient
            loss = compute_loss(y,tx,w)
    
    return w, loss

### Least Squares

In [ ]:
def least_squares(y, tx):
    
    w = np.linalg.solve(tx.T@tx,tx.T@y)
    loss = compute_loss_mse(y, tx, w)
    return w, loss

### Ridge Regression

In [ ]:
def ridge_regression(y, tx, lambda_):

    lambdaI = 2* tx.shape[0] * lambda_ * np.eye(tx.shape[1])
    w = np.linalg.solve(tx.T.dot(tx)+lambdaI,tx.T.dot(y))
    loss = compute_loss_mse(y, tx, w)
    
    return w, loss

### Logistic Regression

In [ ]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    raise ImplementationError
    return w, loss

In [ ]:
def reg_logistic_regression(y, tx, lambda_, intial_w, max_iters, gamma):
    raise Implementation Error
    return w, loss

# III. Implementing the Functions

# IV. Preprocessing the Data